# Building Segmentation

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import tifffile as tiff
import cv2
from skimage.io import imsave, imread, imshow
from skimage import color
from PIL import Image
from skimage.transform import resize
import os

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.metrics import MeanIoU, Accuracy

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    except RuntimeError as e:
        print(e)

In [ ]:
gpus

In [ ]:
# import numpy as np

# def extract_padded_patches(image, patch_size=(512, 512)):
#     height, width = image.shape[:2]
#     patch_height, patch_width = patch_size

#     num_patches_y = (height + patch_height - 1) // patch_height
#     num_patches_x = (width + patch_width - 1) // patch_width

#     # Calculate padding
#     pad_y = num_patches_y * patch_height - height
#     pad_x = num_patches_x * patch_width - width

#     # Pad the image
#     padded_image = np.pad(image, ((0, pad_y), (0, pad_x), (0, 0)), mode='constant')
#     #print(padded_image.shape)
    
#     # Extract patches
#     patches = []

#     for y in range(0, num_patches_y * patch_height, patch_height):
#         for x in range(0, num_patches_x * patch_width, patch_width):
#             patch = padded_image[y:y+patch_height, x:x+patch_width]
#             #plt.imshow(patch/255)
#             patches.append(patch)

#     return np.array(patches)

# #load the .npz file
# def load_images(path,name, labels):
#     npzfile=np.load(path+"{}.npz".format(name))
#     images=npzfile['arr_0']
#     #patches_image=np.zeros()
#     images =[extract_padded_patches(image, (512, 512)) for image in images]
#     images = [patch for patches in images for patch in patches]


#     npzfile=np.load(path+"{}.npz".format(labels))
#     labels=npzfile['arr_0']
#     #labels=[color.rgb2gray(resize(x,(512,512),mode='constant', preserve_range=True)) for x in labels]
    
#     labels =[extract_padded_patches(image, (512, 512)) for image in labels]
#     labels = [patch for patches in labels for patch in patches]
#     labels = np.array(labels)
#     labels=[cv2.threshold(color.rgb2gray(gt), 128, 1, cv2.THRESH_BINARY)[1] for gt in labels]
    
#     return np.array(images),  np.reshape(np.array(labels), (-1, 512, 512, 1))

In [ ]:
# import matplotlib.pyplot as plt

# # Assume 'patches' is a list of images obtained from extract_padded_patches function

# # Create a figure with subplots
# fig, axs = plt.subplots(1, len(patches), figsize=(15, 5))

# # Iterate through patches and display them
# for i, patch in enumerate(patches):
#     axs[i].imshow(patch/255)
#     axs[i].axis('off')
#     axs[i].set_title(f'Patch {i + 1}')

# # Show the plot
# plt.show()


In [ ]:
#load the .npz file
def load_images(path, name, labels):
    npzfile=np.load(path+"{}.npz".format(name))
    images=npzfile['arr_0']


    npzfile=np.load(path+"{}.npz".format(labels))
    labels=npzfile['arr_0']

    return images, labels

In [ ]:
path='/kaggle/input/building-dataset/'



x_train,y_train = load_images(path,'train','train_labels')


x_val,y_val = load_images(path,'val','val_labels')


x_test,y_test = load_images(path,'test','test_labels')


x_train.sort()
y_train.sort()

x_val.sort()
y_val.sort()

x_test.sort()
y_test.sort()

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test,shape, y_test.shape)

In [ ]:
from tensorflow.keras.utils import Sequence
import numpy as np 
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(x_train, y_train, 4)
val_gen = DataGenerator(x_val, y_val, 4)


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
fig.figsize = (30,30)

ax = fig.add_subplot(1,2,1)
ax.imshow(np.reshape(y_train[2]/255,(512,512)),cmap="gray")
plt.axis('off')
plt.title("Ground-truth")

ax = fig.add_subplot(1,2,2)
ax.imshow(x_train[2]/255)
plt.axis('off')
plt.title("Actual image")

# Loss

In [ ]:
from keras import backend as K
from keras.losses import binary_crossentropy
import tensorflow as tf

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + 0.5 * dice_loss(tf.cast(y_true, tf.float32), y_pred)

In [ ]:
import tensorflow as tf

def create_callbacks():
    # Reduce learning rate on plateau
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        mode='min',
        factor=0.5,
        patience=10,
        verbose=1,
        cooldown=1,
        min_delta=0.0001
    )

    # Early stopping
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.0001,
        patience=30,
        verbose=1,
        mode='min',
        restore_best_weights=True
    )

    # Model checkpoint
    # Define the path using os.path.join for cross-platform compatibility
    check_path = os.path.join('.', 'dpp_building.h5')
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        check_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        save_weights_only=True,
        mode='min'
    )

    return [reduce_lr, early_stop, checkpoint]

# Usage example:


In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    # Accuracy plot
    plt.plot(history.history['accuracy'], label='train_acc')
    plt.plot(history.history['val_accuracy'], '--', label='val_acc')
    plt.legend()
    plt.xlabel("No. of epochs")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.show()

    # Loss plot
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], '--', label='val_loss')
    plt.legend()
    plt.xlabel("No. of epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.show()



In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score, precision_score, recall_score

def get_dice(y_true, y_pred):
    intersection = np.sum(y_true * y_pred, axis=(0, 1, 2))
    union = np.sum(y_true**2, axis=(0, 1, 2)) + np.sum(y_pred**2, axis=(0, 1, 2))
    dc = 2 * intersection / union
    return dc


def evaluate_segmentation(y_test, y_pred):
    # Mean Intersection over Union
    mean_iou = MeanIoU(2)
    mean_iou.update_state(y_test, y_pred)
    class_iou = mean_iou.result().numpy()
    mean_iou.reset_states()
    
    # Accuracy
    acc = Accuracy()
    acc.update_state(y_test, y_pred)
    accuracy = acc.result().numpy()
    acc.reset_states()
    
    # Precision
    precision = precision_score(y_test.ravel(), y_pred.ravel(), average='binary')
    
    # Recall
    recall = recall_score(y_test.ravel(), y_pred.ravel(), average='binary')
    
    # Confusion Matrix
    cm = confusion_matrix(y_test.ravel(), y_pred.ravel())
    
    # Classification Report
    cr = classification_report(y_test.ravel(), y_pred.ravel())
    
    # Cohen's Kappa Score
    kappa = cohen_kappa_score(y_test.ravel(), y_pred.ravel())
    
    # Dice Coefficient
    dice_coeff = get_dice(y_test, y_pred)
    mean_dice_coeff = np.mean(dice_coeff)
    
    return {
        "mean_iou": mean_iou,
        "class_iou": class_iou,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "confusion_matrix": cm,
        "classification_report": cr,
        "cohen_kappa_score": kappa,
        "dice_coefficient": dice_coeff,
        "mean_dice_coefficient": mean_dice_coeff
    }


In [ ]:
import matplotlib.pyplot as plt

def visualize_predictions(x_test, y_test, y_pred, num_samples=10):
    fig = plt.figure(figsize=(20, 15))

    for i in range(num_samples):
        # Plot input image
        ax = fig.add_subplot(num_samples, 3, 3*i+1)
        ax.imshow(np.reshape(x_test[i], (512, 512)), cmap="gray")
        plt.axis('off')
        plt.title("Input image")

        # Plot ground truth
        ax = fig.add_subplot(num_samples, 3, 3*i+2)
        ax.imshow(np.reshape(y_test[i], (512, 512)), cmap="gray")
        plt.axis('off')
        plt.title("Ground-truth")

        # Plot predicted image
        ax = fig.add_subplot(num_samples, 3, 3*i+3)
        ax.imshow(np.reshape(y_pred[i], (512, 512)), cmap="gray")
        plt.axis('off')
        plt.title("Predicted image")

    plt.tight_layout()
    plt.show()

# **CTMU_Net**

In [23]:
import tensorflow as tf
class LipPooling(tf.keras.layers.Layer):
    def __init__(self, filters=1):
        super(LipPooling, self).__init__()
        self.conv1=tf.keras.layers.Conv2D(filters, 3, padding='valid')
        self.batch_norm=tf.keras.layers.BatchNormalization()
        self.window_size=(3,3)
    def call(inputs):
        x=self.conv1(inputs)
        x=self.batch_norm(x)
        x=tf.keras.activations.sigmoid(x)
        e_x=tf.keras.activations.exponential(x)
        m_x= inputs*e_x
        e_x=tf.nn.pool(e_x, self.window_size, pooling_type='SUM', padding='VALID')
        m_x=tf.nn.pool(m_x, self.window_size, pooling_type='SUM', padding='VALID')
        x=tf.divide(m_x, e_x+1e-8)

        return x      

In [24]:
def channel_attention(input_feature, ratio=8):
	
	channel_axis = -1
	channel = input_feature._keras_shape[channel_axis]
	
	shared_layer_one = Dense(channel//ratio,
							 activation='relu',
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	shared_layer_two = Dense(channel,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	
	avg_pool = GlobalAveragePooling2D()(input_feature)    
	avg_pool = Reshape((1,1,channel))(avg_pool)
	assert avg_pool._keras_shape[1:] == (1,1,channel)
	avg_pool = shared_layer_one(avg_pool)
	assert avg_pool._keras_shape[1:] == (1,1,channel//ratio)
	avg_pool = shared_layer_two(avg_pool)
	assert avg_pool._keras_shape[1:] == (1,1,channel)
	
	max_pool = GlobalMaxPooling2D()(input_feature)
	max_pool = Reshape((1,1,channel))(max_pool)
	assert max_pool._keras_shape[1:] == (1,1,channel)
	max_pool = shared_layer_one(max_pool)
	assert max_pool._keras_shape[1:] == (1,1,channel//ratio)
	max_pool = shared_layer_two(max_pool)
	assert max_pool._keras_shape[1:] == (1,1,channel)
	
	cbam_feature = Add()([avg_pool,max_pool])
	cbam_feature = Activation('sigmoid')(cbam_feature)
	
	# if K.image_data_format() == "channels_first":
	# 	cbam_feature = Permute((3, 1, 2))(cbam_feature)
	
	return multiply([input_feature, cbam_feature])

In [38]:
#CTM module

import tensorflow as tf

def get_row_attention(q, k, f):
  """
  Calculates row attention based on Q, K, and F feature maps.

  Args:
    q: A tensor of shape (batch_size, height, width, reduced_channels).
    k: A tensor of the same shape as q.
    f: A tensor of shape (batch_size, height, width, channels).

  Returns:
    A tensor of shape (batch_size, height, width, channels).
  """
  
  # Get dimensions
  batch_size, height, width, reduced_channels = q.shape
  channels = f.shape[-1]

  # Calculate correlations
  correlations = tf.matmul(q, tf.transpose(k))  # (batch_size, H, W, H)
  correlations = tf.nn.softmax(correlations, axis=-1)  # (batch_size, H, W, H)

  # Generate V feature map
  v = tf.keras.layers.Conv2D(channels, kernel_size=1, activation=None, padding='same')(f)

  # Calculate row attention scores
  row_attention = []
  for i in range(height):
    # Extract relevant feature vectors for row i
    qi = q[:, i, :, :]  # (batch_size, 1, W, reduced_channels)
    ki = k[:, i, :, :]  # (batch_size, W, reduced_channels)
    vi = v[:, i, :, :]  # (batch_size, 1, W, channels)
    phi_i = k[:, :, i, :]  # (batch_size, W, channels)

    # Calculate attention for row i
    row_i_attention = tf.reduce_sum(correlations[:, i, :, :] * phi_i, axis=1) + vi
    row_attention.append(row_i_attention)
  row_attention = tf.stack(row_attention, axis=1)  # (batch_size, H, W, channels)

  return row_attention

# # Example usage (assuming Q, K, and F are defined elsewhere)
# row_attention = get_row_attention(q, k, f)
# print(f"Row attention shape: {row_attention.shape}")


In [26]:
import tensorflow as tf

def get_column_attention(yrow, q, k, f):
  """
  Calculates column attention based on Yrow, Q, K, and F feature maps.

  Args:
    yrow: A tensor of shape (batch_size, height, width, channels).
    q: A tensor of the same shape as yrow.
    k: A tensor of the same shape as q.
    f: A tensor of shape (batch_size, height, width, channels).

  Returns:
    A tensor of shape (batch_size, height, width, channels).
  """
  
  # Get dimensions
  batch_size, height, width, channels = yrow.shape

  # Calculate correlations
  correlations = tf.matmul(yrow, tf.transpose(k, perm=[0, 2, 1, 3]))  # (batch_size, H, W, C)
  correlations = tf.nn.softmax(correlations, axis=-1)  # (batch_size, H, W, C)

  # Calculate column attention scores
  column_attention = []
  for i in range(width):
    # Extract relevant feature vectors for column i
    yrow_i = yrow[:, :, i, :]  # (batch_size, H, 1, channels)
    ki = k[:, :, i, :]  # (batch_size, H, channels)
    fi = f[:, :, i, :]  # (batch_size, H, 1, channels)
    phi_col_i = k[:, :, :, i]  # (batch_size, H, channels)

    # Calculate attention for column i
    col_i_attention = tf.reduce_sum(correlations[:, :, i, :] * phi_col_i, axis=-1) + fi
    column_attention.append(col_i_attention)
  column_attention = tf.stack(column_attention, axis=2)  # (batch_size, H, W, channels)

  return column_attention

# # Example usage (assuming Yrow, Q, K, and F are defined elsewhere)
# column_attention = get_column_attention(yrow, q, k, f)
# print(f"Column attention shape: {column_attention.shape}")


In [39]:
input_shape = (4, 512, 512, 3)
inputs= tf.keras.Input(shape=(512, 512, 3))
# row attention
x=tf.keras.layers.Conv2D(16,1, activation='relu', padding='same',input_shape=input_shape[1:])(inputs)

q=tf.keras.layers.Conv2D(16,1, activation='relu', padding='same')(x)
k=q
f=x
yrow= get_row_attention(q,k,f)
# column attention
q=tf.keras.layers.Conv2D(512,1, activation='relu', padding='same')(yrow)
k=q
f=yrow
output= get_column_attention(yrow,q,k,f)

model=tf.keras.Model(inputs=inputs, outputs=output)

model.summary()

ValueError: Exception encountered when calling layer "tf.linalg.matmul_9" (type TFOpLambda).

Dimensions must be equal, but are 16 and 512 for '{{node tf.linalg.matmul_9/MatMul}} = BatchMatMulV2[T=DT_FLOAT, adj_x=false, adj_y=false](Placeholder, Placeholder_1)' with input shapes: [?,512,512,16], [16,512,512,?].

Call arguments received:
  • a=tf.Tensor(shape=(None, 512, 512, 16), dtype=float32)
  • b=tf.Tensor(shape=(16, 512, 512, None), dtype=float32)
  • transpose_a=False
  • transpose_b=False
  • adjoint_a=False
  • adjoint_b=False
  • a_is_sparse=False
  • b_is_sparse=False
  • output_type=None
  • name=None

In [20]:
?tf.keras.layers.Conv2D

Init signature:
tf.keras.layers.Conv2D(
    filters,
    kernel_size,
    strides=(1, 1),
    padding='valid',
    data_format=None,
    dilation_rate=(1, 1),
    groups=1,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs,
)
Docstring:     
2D convolution layer (e.g. spatial convolution over images).

This layer creates a convolution kernel that is convolved
with the layer input to produce a tensor of
outputs. If `use_bias` is True,
a bias vector is created and added to the outputs. Finally, if
`activation` is not `None`, it is applied to the outputs as well.

When using this layer as the first layer in a model,
provide the keyword argument `input_shape`
(tuple of integers or `None`, does not include the sample axis),
e.g. `input_shape=(128, 128, 3)` for 128x128 RGB picture

In [13]:
def extract_patches(feature_map, patch_size=4):
    patches=tf.image.extract_patches(images=feature_map, 
                                     sizes=[1, patch_size, patch_size,1],
                                     strides=[1, patch_size, patch_size,1],
                                    padding='VALID')
    return patches

# Re-A Net

In [21]:
class resConv(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(resConv, self).__init__()
        self.conv1=tf.keras.layers.Conv2D(filters, 3, padding='valid')
    
    def call(inputs):
        x=self.conv1(inputs)
        x=tf.nn.activation.relu(x)
        x=self.conv1(x)
        x=tf.nn.activation.relu(x)
        x=self.conv1(inputs)
        x+=inputs
        x=tf.nn.activation.relu(x)
        return x

In [23]:
import tensorflow as tf
inputs=tf.keras.layers.Input(shape=(512,512,3))

x=tf.keras.layers.Conv2D(64,3, padding='same')(inputs)
resConvLayer=resConv(64)
x=resConvLayer(x)
resConvLayer=resConv(128)
resConvLayer(x)
resConvLayer=resConv(256)
resConvLayer(x)
resConvLayer=resConv(512)
resConvLayer(x)


model=Model(inputs=inputs, outputs=x)
model.summary()

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__call() takes 1 positional argument but 2 were given


In [ ]:
model=DPM_UNet((512,512,3))

In [ ]:
model.summary()

In [ ]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', iou_coef])

In [ ]:
callbacks = create_callbacks()
history = model.fit(train_gen, validation_data=val_gen, batch_size=4,shuffle=True, verbose=1
                                  ,epochs = 100, callbacks = callbacks )


In [ ]:
plot_history(history)

In [ ]:
y_pred = dppnetmodel.predict(x_test)
y_pred = y_pred>0.5

In [ ]:
visualize_predictions(x_test, y_test, y_pred, num_samples=9)

In [ ]:
model.load_weights('C:\\Users\\Admin\\Documents\\road_segmentation\\dpp_ROAD.h5')

In [ ]:
y_predicted=[]
for i in range(len(x_test)):
    image= np.expand_dims(x_test[i], axis=0)

    y_pred = dppnetmodel.predict(image)
    y_pred=y_pred>0.5
    y_predicted.append(y_pred)



In [ ]:
from sklearn.metrics import *
gt=(y_test.ravel()).astype('int')
pd=(np.array(y_predicted).ravel()).astype('int')
f1 = f1_score(gt,pd,average='macro')
kappa = cohen_kappa_score(gt,pd)
accuracy = accuracy_score(gt,pd)
jaccard = jaccard_score(gt,pd,average='macro')
precision = precision_score(gt,pd,average='macro')
recall = recall_score(gt,pd,average='macro')
print(np.unique(gt),np.unique(pd))
print("F1 SCORE:", f1)
print("Kappa:",kappa)
print("Accuracy:",accuracy)
print("Jaccard Score:",jaccard)
print("Precision:",precision)
print("Recall:",recall)
